In [29]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [30]:
names = ['S' + str(format(i, '02')) for i in range(6)] # + 1 because range starts at 0
names.pop(0) # Pop the first element of the list to match the names of the files
print(names)

# Create a list of the variables and its columns
variables = ['EDA', 'HR', 'IBI','TEMP']
final_columns = {
    #'ACC': ['x', 'y', 'z'],
    'EDA': ['EDA'],
    'HR': ['HR',],
    'IBI': ['IBI', 'IBI_ms'],
    'TEMP': ['TEMP'],
}
print(variables)

['S01', 'S02', 'S03', 'S04', 'S05']
['EDA', 'HR', 'IBI', 'TEMP']


In [31]:
df_HR = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S01/HR.csv',header=None)
df_HR.head()
start_time = df_HR.iloc[0,0]
df2 = df_HR.T
init = df2.pop(0)
hz = df2.pop(1)
df_HR = df2.T
time = [start_time+(i*1000) for i in range(len(df_HR))]
df_HR['time'] = time
df_HR.set_index('time', inplace=True)
df_HR = df_HR.rename(columns={0: 'HR'}, inplace=False)
df_HR

,HR
time,
1.644226e+09,83.00
1.644227e+09,83.00
1.644228e+09,72.67
1.644229e+09,87.00
1.644230e+09,80.00
...,...
1.647471e+09,83.35
1.647472e+09,83.33
1.647473e+09,83.28


In [32]:
# Import timelogs for visualization
# This DataFrame contains the time logs for each subject responses
timelogs = pd.read_csv('Time_logs.csv')
timelogs = timelogs.drop(0, axis=0)
timelogs = timelogs.drop(['Session ID', 'Unnamed: 24'], axis=1)
timelogs

,S. ID.,Start Time,End Time,Session ID,Consent,Unnamed: 5,Baseline/Questionniare,Unnamed: 7,Stroop Test,Unnamed: 9,...,Hyperventilation,Unnamed: 17,Relax.2,Unnamed: 19,Questionniare,Unnamed: 21,Relax/Baseline,Unnamed: 23,Unnamed: 24,Date
0,NaN,NaN,NaN,NaN,Start,End,Start,End,Start,End,...,Start,End,Start,End,Start,End,Start,End,NaN,NaN
1,S01,9:22,10:22,1445405.0,9:22,9:27,9:39,9:43,9:29,9:34,...,10:00,10:02,10:02,10:07,10:08,10:12,10:12,10:22,Day 1,7-Feb-2022
2,S02,9:48,10:51,1445449.0,9:48,9:50,9:52,10:02,10:03,10:08,...,10:29,10:31,10:31,10:36,10:36,10:41,10:41,10:51,Day 1,7-Feb-2022
3,S03,10:50,11:50,1445450.0,10:50,10:53,10:55,11:02,11:04,11:09,...,11:28,11:30,11:30,11:35,11:35,11:40,11:40,11:50,Day 1,7-Feb-2022
4,S04,11:25,12:28,1445452.0,11:25,11:30,11:30,11:40,11:40,11:45,...,12:05,12:07,12:07,12:12,12:12,12:18,12:18,12:28,Day 1,7-Feb-2022
5,S05,9:05,10:06,1449998.0,9:05,9:10,9:12,9:22,9:23,9:29,...,9:49,9:51,9:51,9:56,9:56,10:01,10:01,10:06,Day 2,14-Feb-2022
6,S06,9:41,10:40,1450001.0,9:41,9:43,9:44,9:54,9:55,10:01,...,10:23,10:25,10:25,10:30,10:30,10:35,10:35,10:40,Day 2,14-Feb-2022
7,S07,10:25,11:26,1450002.0,10:25,10:29,10:31,10:41,10:43,10:48,...,11:09,11:11,11:11,11:16,11:16,11:21,11:21,11:26,Day 2,14-Feb-2022
8,S08,12:08,1:05,1450061.0,12:08,12:10,12:12,12:22,12:23,12:28,...,12:48,12:50,12:50,12:55,12:55,1:00,1:00,1:05,Day 2,14-Feb-2022
9,S09,12:34,1:31,1450063.0,12:34,12:36,12:38,12:48,12:49,12:54,...,1:14,1:16,1:16,1:21,1:21,1:26,1:26,1:31,Day 2,14-Feb-2022


In [41]:
# Convert the time logs to unix timestamps
# This is done to be able to compare the time logs with the data
for column in timelogs.columns:
    unix_timestamps = []
    if column != 'Date' and column != 'S. ID.':
        timelogs[column] = timelogs.Date + ' ' + timelogs[column]
        times = timelogs[column].values.tolist()
        for date in times:
            dt_obj = datetime.strptime(date, '%d-%b-%Y %H:%M')
            unix_timestamps.append(int(dt_obj.timestamp()))
        timelogs[column] = unix_timestamps
timelogs = timelogs.drop(['Date', 'Start Time', 'End Time'], axis=1)
timelogs

,S. ID.,Start Time,End Time,Consent,Unnamed: 5,Baseline/Questionniare,Unnamed: 7,Stroop Test,Unnamed: 9,Relax,...,Unnamed: 15,Hyperventilation,Unnamed: 17,Relax.2,Unnamed: 19,Questionniare,Unnamed: 21,Relax/Baseline,Unnamed: 23,Date
1,S01,1644225720,1644229320,1644225720,1644226020,1644226740,1644226980,1644226140,1644226440,1644226440,...,1644227940,1644228000,1644228120,1644228120,1644228420,1644228480,1644228720,1644228720,1644229320,7-Feb-2022
2,S02,1644227280,1644231060,1644227280,1644227400,1644227520,1644228120,1644228180,1644228480,1644228480,...,1644229740,1644229740,1644229860,1644229860,1644230160,1644230160,1644230460,1644230460,1644231060,7-Feb-2022
3,S03,1644231000,1644234600,1644231000,1644231180,1644231300,1644231720,1644231840,1644232140,1644232140,...,1644233280,1644233280,1644233400,1644233400,1644233700,1644233700,1644234000,1644234000,1644234600,7-Feb-2022
4,S04,1644233100,1644236880,1644233100,1644233400,1644233400,1644234000,1644234000,1644234300,1644234300,...,1644235500,1644235500,1644235620,1644235620,1644235920,1644235920,1644236280,1644236280,1644236880,7-Feb-2022
5,S05,1644829500,1644833160,1644829500,1644829800,1644829920,1644830520,1644830580,1644830940,1644830940,...,1644832140,1644832140,1644832260,1644832260,1644832560,1644832560,1644832860,1644832860,1644833160,14-Feb-2022
6,S06,1644831660,1644835200,1644831660,1644831780,1644831840,1644832440,1644832500,1644832860,1644832920,...,1644834180,1644834180,1644834300,1644834300,1644834600,1644834600,1644834900,1644834900,1644835200,14-Feb-2022
7,S07,1644834300,1644837960,1644834300,1644834540,1644834660,1644835260,1644835380,1644835680,1644835680,...,1644836880,1644836940,1644837060,1644837060,1644837360,1644837360,1644837660,1644837660,1644837960,14-Feb-2022
8,S08,1644840480,1644800700,1644840480,1644840600,1644840720,1644841320,1644841380,1644841680,1644841680,...,1644842880,1644842880,1644843000,1644843000,1644843300,1644843300,1644800400,1644800400,1644800700,14-Feb-2022
9,S09,1644842040,1644802260,1644842040,1644842160,1644842280,1644842880,1644842940,1644843240,1644843240,...,1644801240,1644801240,1644801360,1644801360,1644801660,1644801660,1644801960,1644801960,1644802260,14-Feb-2022
10,S10,1644801060,1644804300,1644801060,1644801180,1644801240,1644801780,1644801780,1644802080,1644802140,...,1644803280,1644803340,1644803460,1644803460,1644803760,1644803760,1644803940,1644803940,1644804300,14-Feb-2022
